In [27]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict,Literal,Annotated
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage,HumanMessage
from langchain.prompts import PromptTemplate
import os
from dotenv import load_dotenv
load_dotenv()

True

In [28]:
api_key=os.getenv("GEMINI_KEY")
llm=ChatGoogleGenerativeAI(model="gemini-2.5-flash",api_key=api_key) 

In [34]:
class tweetState(TypedDict):
    topic:str
    tweet:str
    evaluation:Literal["approved","not approved"]
    iteration:int
    max_iteration:int

In [30]:
def generate_tweet(state:tweetState):
    topic=state["topic"]
    messages=[SystemMessage(content="You are a funny Twitter/X influencer"),
              HumanMessage(content=f'''
write a short, original and hilarious tweet on the topic: {topic}
Rules:
- Do Not use question answer format.
- Max. 280 characters.
- use Observational, Meta Joke, or Pop culture references 
''')]
    
    response=llm.invoke(messages)
    return {"tweet":response.content}

def evaluate_tweet(state:tweetState):
    tweet=state["tweet"]
    template='''
based on the given tweet: {tweet} Evaluate it properly
on the basis following rules 
Rules:
- Do Not use question answer format.
- Max. 280 characters.
- use Observational, Meta Joke, or Pop culture references

and provide result in either 'approved' or 'not approved'
'''
    prompt=PromptTemplate(template=template,input_variables=["tweet"])

    chain=prompt|llm
    result=chain.invoke({"tweet":tweet})

    return {"evaluation":result.content}

def optimize_tweet(state:tweetState):
    tweet=state["tweet"]
    template='''Improve the following tweet:{tweet}, make it funny, Pop cultured'''
    prompt=PromptTemplate(template=template,input_variables=["tweet"])
    chain=prompt|llm
    result=chain.invoke({"tweet":tweet})
    iteration=state["iteration"]+1
    return {"tweet":result,"iteration":iteration}
    


In [31]:
def check_tweet(state:tweetState):
    if state["evaluation"]=="approved" or state["iteration"]>state["max_iteration"]:
        return "approved"
    else:
        return "not approved"

In [32]:
graph=StateGraph(tweetState)

graph.add_node("generate_tweet",generate_tweet)
graph.add_node("evaluate_tweet",evaluate_tweet)
graph.add_node("optimize_tweet",optimize_tweet)

graph.add_edge(START,"generate_tweet")
graph.add_edge("generate_tweet","evaluate_tweet")

graph.add_conditional_edges("evaluate_tweet",check_tweet,{'approved':END,'not approved':"optimize_tweet"})
graph.add_edge("optimize_tweet","evaluate_tweet")

workflow=graph.compile()

In [ ]:
result=workflow.invoke({"topic":"Government","iteration":1,"max_iteration":2})